In [ ]:
# First, install all libraries here
%pip install numpy requests pandas bs4

In [ ]:
# Import all necessary libraries for this workflow to run

import requests
import pandas as pd
from bs4 import BeautifulSoup
import os
import zipfile
from glob import glob

In [ ]:
# Gather all experimental PTM data from dbPTM and download them

BASE_URL = 'https://biomics.lab.nycu.edu.tw/dbPTM' # This is for dbPTM 2025 - I wish dbPTM 2022 URL was up as well for empirical comparison

response = requests.get(f'{BASE_URL}/download.php')
files_to_download = []

if response.ok:
    soup = BeautifulSoup(response.text, 'html.parser')
    for a in soup.find_all('a', href=True):
        href = a['href']
        if 'experiment' in href and href.endswith('.zip'):
            files_to_download.append(f"{BASE_URL}/{href}")

if not os.path.exists('./dbptm_data'):
    os.mkdir('dbptm_data')

all_files = []

for file in files_to_download:
    actual_file = file.split('/')[-1]
    with requests.get(file, stream=True) as response:
        with open(f"dbptm_data/{actual_file}", 'wb') as f:
            for chunk in response.iter_content(chunk_size=4096):
                f.write(chunk)
    all_files.append(actual_file)
    print("Downloaded", actual_file)

In [ ]:
# Extract contents from the downloaded .zip files
for file in all_files:
    with zipfile.ZipFile(f'dbptm_data/{file}', 'r') as z_f:
        z_f.extractall(f'dbptm_data')

In [ ]:
# With all of the files extracted, we start working on it.
# You can see these files are just "FILE" files.
# They're supposed to be ".tsv" files (tab-separated values).
# Parsing isn't particularly difficult, but it's cumbersome to deal with.

# We extract all PTMs to deal with
PTMS = [file.split('.')[0] for file in all_files]

# This function is used to capture the 21-residue sequence window from the entire protein sequence.
# Works pretty well (I think)
def construct_subsequence(protein: str, site: int, no_stream_aa: int = 10):
    # Treat PTM site as 0-based (so subtract 1 for convenience)
    start = max(0, site - no_stream_aa)
    end = min(len(protein), site + no_stream_aa + 1)
    subsequence: str = protein[start:end]
    if site < no_stream_aa:
        subsequence = ('-' * (no_stream_aa - site)) + subsequence 
    if site + no_stream_aa > len(protein):
        subsequence += ('-' * ((site + no_stream_aa + 1) - len(protein)))
    
    return subsequence


# Run this function for each PTM file to get protein sequences for all proteins in each PTM file which are not given
def preprocess(filepath: str):
    # Manual label of columns for reference
    df = pd.read_csv(
        filepath,
        names=[
            'ProID',
            'Acc#',
            'ModSite',
            'PTM',
            'EvdId',
            'Seq'
        ],
        header=None,
        sep='\t'
    )

    # Extract all rows which have no sequence windows
    na_df = df[df['Seq'].isna()]
    if len(na_df):
        # If there is at least one row, we use that to process.
        print("For file", filepath.split('/')[-1])
        # This is a point of failure. Thankfully, no such empty sequence window exists for which no UniProt Accession Number is given
        # Also, unique called - can't have the API handle 
        accession_numbers = na_df['Acc#'].unique().tolist()
        for accession_number in accession_numbers:
            got_response = True
            got_sequence = True
            while got_response:
                try:
                    got_response = False
                    # Fetch response from EBI API.
                    response = requests.get( # I understand doing this asynchronously would have benefitted majorly, but I was too lazy to handle them
                        f'http://www.ebi.ac.uk/proteins/api/proteins/{accession_number}' # Also, thankfully, much less protein sequences to deal with
                    )
                    print(f"\tGot response for {accession_number}.")
                except:
                    got_response = True
                try:
                    # Would have preferred calling ".get()", but this works too I guess...
                    sequence = response.json()['sequence']['sequence']
                except:
                    got_sequence = False
                    sequence = ''
                    print("No sequence found for", accession_number)
            
            # Some processing where we get all recorded mod sites for that protein
            positions = na_df[na_df['Acc#'] == accession_number].ModSite.to_list()
            for position in positions:
                # If the sequence was retrieved, we extract the 21-residue sequence window relative to the mod site.
                if got_sequence:
                    subsequence = construct_subsequence(sequence, position-1)
                else:
                    subsequence = ''
                df.loc[(df['Acc#'] == accession_number) & (df['ModSite'] == position), 'Seq'] = subsequence

        # Going to save the new file as a CSV (maybe not a good idea, but at no point would you actually have commas in dbPTM's data)
        df.to_csv(f"{filepath}.csv", index=False)
    else:
        # Just save it as is, no alterations needed for now.
        df.to_csv(f"{filepath}.csv", index=False)
        print("Clear for", filepath.split('/')[-1])

In [ ]:
# Should have removed zip files, but kept them for restoration purposes
all_files = [i.split('.')[0] for i in glob('dbptm_data/*.zip')]
for file in all_files:
    preprocess(file)

In [ ]:
# Now in all honesty, this is a pretty stupid function, but there's a reason why it's so absurd
# This is all done in the case where the EBI API was unable to fetch the protein sequence from the Accession Number
# So we think there are a few possibilities:
# 1. First, just query the same accession number using rest.uniprot instead of the EBI API 
#    If the protein sequence is found, huzzah! Otherwise:
# 2. Check why no protein sequence exists
# 2.1. In the case of the protein sequence being deleted, fetch the protein's UNIPARC ID and use that to query to ANOTHER URL for the protein sequence - this always works, so no need to hypervise
# 2.2. In the case of the protein sequence being demerged, get the first available accession number in the merged list and use that accession number to make a request and get the protein sequence
def preprocess_again(filepath: str):
    df = pd.read_csv(
        filepath
    )

    na_df = df[df['Seq'].isna()]

    if len(na_df):
        print("Doing for", filepath.split('/')[-1])
        accession_numbers = na_df['Acc#'].unique().tolist()
        for accession_number in accession_numbers:
            got_response = True
            while got_response:
                try:
                    got_response = False
                    response = requests.get(
                        f'https://rest.uniprot.org/uniprotkb/{accession_number}',
                        headers={
                            'Accept': 'application/json'
                        }
                    )
                    print("\tGot response for", accession_number)
                except Exception as e:
                    got_response = True
            got_sequence = True
            try:
                response: dict = response.json()
                if 'inactiveReason' in response.keys():
                    if response['inactiveReason']['inactiveReasonType'] == 'DELETED':
                        print("\tWas deleted")
                        uniparc_id = response['extraAttributes']['uniParcId']
                        print(uniparc_id)
                        got_response = True
                        while got_response:
                            try:
                                got_response = False
                                response: dict = requests.get(
                                    f'https://rest.uniprot.org/uniparc/{uniparc_id}',
                                    headers={
                                        'Accept': 'application/json'
                                    }
                                )
                            except Exception as e:
                                got_response = True
                    elif response['inactiveReason']['inactiveReasonType'] == 'DEMERGED':    
                        print("\tWas merged")
                        new_accession_number = response['inactiveReason']['mergeDemergeTo'][0]
                        print(new_accession_number)
                        got_response = True
                        while got_response:
                            try:
                                got_response = False
                                response: dict = requests.get(
                                    f'https://rest.uniprot.org/uniprotkb/{new_accession_number}',
                                    headers={
                                        'Accept': 'application/json'
                                    }
                                )
                            except Exception as e:
                                got_response = True
                    print("Retrieved check")
                    print(response.json()['sequence']['value'])
                    print("Clearance check")
                    sequence = response.json()['sequence']['value']
                else:
                    sequence = response['sequence']['value']
            except Exception as e:
                sequence = ''
                got_sequence = False
                print("\t\tNo sequence found for", accession_number)
                
            positions = na_df[na_df['Acc#'] == accession_number].ModSite.to_list()
            for position in positions:
                if got_sequence:
                    subsequence = construct_subsequence(sequence, position-1)
                else:
                    subsequence = ''
                df.loc[(df['Acc#'] == accession_number) & (df['ModSite'] == position), 'Seq'] = subsequence
        # Overwrite the information
        df.to_csv(f"{filepath}.csv", index=False)
        
        return 1
    else:
        print("Nothing for", filepath.split('/')[-1])
        return 0


In [ ]:
# Same deal again
all_files = [i for i in glob('dbptm_data/*.csv')]
for file in all_files:
    preprocess_again(file)

In [ ]:
# This is actually stupid, don't try this, just edit the above funtion
# This just fetches the missing accession numbers
def preprocess_again_again(filepath: str):
    df = pd.read_csv(
        filepath
    )

    na_df = df[df['Acc#'].isna()]

    if len(na_df):
        print("Doing for", filepath.split('/')[-1])
        accession_numbers = na_df['ProID'].unique().tolist()
        for accession_number in accession_numbers:
            got_response = True
            while got_response:
                try:
                    got_response = False
                    response = requests.get(
                        f'https://rest.uniprot.org/uniprotkb/{accession_number}',
                        headers={
                            'Accept': 'application/json'
                        }
                    )
                    print("\tGot response for", accession_number)
                except Exception as e:
                    got_response = True
            got_sequence = True
            try:
                response: dict = response.json()
                if 'inactiveReason' in response.keys():
                    if response['inactiveReason']['inactiveReasonType'] == 'DELETED':
                        print("\tWas deleted")
                        uniparc_id = response['extraAttributes']['uniParcId']
                        print(uniparc_id)
                        got_response = True
                        while got_response:
                            try:
                                got_response = False
                                response: dict = requests.get(
                                    f'https://rest.uniprot.org/uniparc/{uniparc_id}',
                                    headers={
                                        'Accept': 'application/json'
                                    }
                                )
                            except Exception as e:
                                got_response = True
                    elif response['inactiveReason']['inactiveReasonType'] == 'DEMERGED':    
                        print("\tWas merged")
                        new_accession_number = response['inactiveReason']['mergeDemergeTo'][0]
                        print(new_accession_number)
                        got_response = True
                        while got_response:
                            try:
                                got_response = False
                                response: dict = requests.get(
                                    f'https://rest.uniprot.org/uniprotkb/{new_accession_number}',
                                    headers={
                                        'Accept': 'application/json'
                                    }
                                )
                            except Exception as e:
                                got_response = True
                    print("Retrieved check")
                    print(response.json()['sequence']['value'])
                    print("Clearance check")
                    sequence = response.json()['sequence']['value']
                else:
                    sequence = response['sequence']['value']
            except Exception as e:
                sequence = ''
                got_sequence = False
                print("\t\tNo sequence found for", accession_number)
                
            positions = na_df[na_df['Acc#'] == accession_number].ModSite.to_list()
            for position in positions:
                if got_sequence:
                    subsequence = construct_subsequence(sequence, position-1)
                else:
                    subsequence = ''
                df.loc[(df['Acc#'] == accession_number) & (df['ModSite'] == position), 'Seq'] = subsequence
        # Overwrite the information
        df.to_csv(f"{filepath}.csv", index=False)
        
        return 1
    else:
        print("Nothing for", filepath.split('/')[-1])
        return 0

In [ ]:
# Once more
all_files = [i for i in glob('dbptm_data/*.csv')]
for file in all_files:
    preprocess_again(file)

In [ ]:
# With all of that done, we can finally move on to the next step
import typing
import json
import numpy as np

# Make a list of residues/amino acids
AA = "A C D E F G H I K L M N P Q R S T V W Y".split(' ')

# Now we have this function that constructs positional frequency matrices.
# You can calculate by frequency, by natural log frequency, or any of the base 2 or base 10 frequencies.
# It's defaulted to 'freq' but you can change it below
# Also this function takes in all filenames instead of a single one
# Thought it better to do this way
def calculate_values(filenames: list[str]):
    # For each PTM
    for filename in filenames:
        print("For", filename)
        df = pd.read_csv(
            filename,
        )
        df['mid'] = df['Seq'].str[10]
        sub_dfs = [group for _, group in df.groupby('mid')]
        # For each residue/amino acid
        for sub_df in sub_dfs:
            # Sanity check - the dataset was pretty unclean
            # Get all sequences where the mod site is said amino acid
            sequences = sub_df['Seq'].to_list()
            # Transpose them
            transposed = list(map(''.join, zip(*sequences)))
            # Construct relative positions
            pos = [f"+{i}" if i > 0 else f"{i}" for i in range(-10, 11)]
            # Create a table based on relative position per amino acid
            table = {i: seq for i, seq in zip(pos, transposed)}

            def log_odds(sequences: dict, method: typing.Literal['freq', 'log-e', 'log10', 'log2'] = 'freq') -> dict[dict[str, float]]:
                table = dict.fromkeys(sequences.keys(), 0)

                for position, sequence in sequences.items():
                    log_odds_table = dict.fromkeys(AA, 0)
                    for aa in AA:
                        # Formula is count of an amino acid 'aa' over count of all amino acids in the transposed sequence (or basically length of a transposed array of protein sequences)
                        total = len(sequence)
                        count = sequence.count(aa)
                        log_odds_table[aa] = count / total # For now, let's keep it simple count_a / count
                        # And then apply NumPy-compatible logarithms
                        if method == 'log-e':
                            log_odds_table[aa] = np.log(log_odds_table[aa])
                        elif method == 'log10':
                            log_odds_table[aa] = np.log10(log_odds_table[aa])
                        elif method == 'log2':
                            log_odds_table[aa] = np.log2(log_odds_table[aa])
                        
                        if log_odds_table[aa] == float('-inf'):
                            log_odds_table[aa] = '-inf'

                    table[position] = log_odds_table

                return table

            method = 'freq' # CHANGE METHOD HERE
            # Calculate matrices for the table
            table = log_odds(table, method)

            print("\tCalculated against", sub_df.iloc[0]['mid'])
            if not os.path.exists(f"tables/{filename.split('.')[0]}"):
                os.mkdir(f"tables/{filename.split('.')[0]}")
            if not os.path.exists(f"tables/{filename.split('.')[0]}/{method}"):
                os.mkdir(f"tables/{filename.split('.')[0]}/{method}")
            with open(f"tables/{filename.split('.')[0]}/{method}/{sub_df.iloc[0]['mid']}.json", 'w') as f:
                json.dump(table, f, indent=2)
            # save_table(pd.DataFrame(table), filename)
        print("Calculated for", filename.split('/')[-1].split('.')[0])

In [ ]:
# Okay, NOW once more - for real this time
all_files = [i for i in glob('dbptm_data/*.csv')]
calculate_values(all_files)

##### And there you have it! The tables have been constructed using a clean and processed version of dbPTM data.